In [0]:
dbutils.notebook.run("../Mounting_storage", 120)

In [0]:
dbutils.fs.ls("/mnt/blobstorage")

In [0]:
drivers_schema = """
driverId INT,
driverRef STRING,
number INT,
code STRING,
name STRUCT<forename: STRING, surname: STRING>,
dob DATE,
nationality STRING,
url STRING
""" 

In [0]:
name_schema = """
forename STRING,
surname STRING
"""

In [0]:
df_drivers = spark.read.json("/mnt/blobstorage/drivers.json", schema = drivers_schema)
display(df_drivers)

In [0]:
df_drivers.printSchema()

### Removing columns and add new columns

In [0]:
from pyspark.sql.functions import col, concat, current_timestamp, lit

In [0]:
drivers_with_columns_df = df_drivers.withColumnRenamed("driverId","driver_id")\
    .withColumnRenamed("driverRef","driver_ref")\
        .withColumn("ingestion_time", current_timestamp())\
            .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname")))

### Dropping unwanted column

In [0]:
df_drivers_final = drivers_with_columns_df.drop(col("url"))

In [0]:
display(df_drivers_final)

### write it to parquet format

In [0]:
df_drivers_final.write.mode("overwrite").parquet("/mnt/blobstorage/drivers")

In [0]:
%fs
ls mnt/blobstorage/drivers